# Lab: Reusing a Model

Here we are going to load a previously trained model and use it 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elephantscale/cool-ML-demos/blob/main/image/img5-reusing-a-model.ipynb)

### Runtime
15 minutes



In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

In [ ]:
## Loading our custom utils files
import os
import sys
from pathlib import Path

# Hack to download image utils when running on Colab ..etc
import os
import urllib.request

file_url = 'https://raw.githubusercontent.com/elephantscale/es-public/master/deep-learning/image_utils.py'
file_location = "image_utils.py"

if not os.path.exists (file_location):
    file_location = os.path.basename(file_location)
    if not os.path.exists(file_location):
        print("Downloading : ", file_url)
        urllib.request.urlretrieve(file_url, file_location)
print('file_location:', file_location)

## TF-GPU Debug
The following block tests if TF is running on GPU.

In [ ]:
## This block is to tweak TF running on GPU
## You may comment this out, if you are not using GPU

## ---- start Memory setting ----
## Ask TF not to allocate all GPU memory at once.. allocate as needed
## Without this the execution will fail with "failed to initialize algorithm" error

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

## Step 1: Download Data



In [ ]:
## --- Cats-Dogs

BATCH_SIZE=128
IMG_HEIGHT = 150
IMG_WIDTH = 150
SAVED_MODEL_FILE='cat-dog-model.h5'

import os

data_location = 'https://elephantscale-public.s3.amazonaws.com/data/images/cat-dog-redux.zip'

data_location_local = keras.utils.get_file(fname=os.path.basename(data_location),
                                           origin=data_location, extract=True)
print ('local download file: ', data_location_local)
data_dir = os.path.join(os.path.dirname(data_location_local), 'cat-dog-redux')
print ('local data dir: ', data_dir)
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'val')
print ('train dir:', train_dir)
print ('validation dir:', validation_dir)




In [ ]:
## Horses or Humans

# BATCH_SIZE=32
# IMG_HEIGHT = 300
# IMG_WIDTH = 300
# SAVED_MODEL_FILE='horse-human-model.h5'

# import os

# data_location = ' https://elephantscale-public.s3.amazonaws.com/data/images/horse-or-human.zip'
# data_location_local = keras.utils.get_file(fname=os.path.basename(data_location),
#                                            origin=data_location, extract=True)
# print ('local download file: ', data_location_local)

# ## Peek inside the directory
# # download_dir = os.path.dirname(data_location_local)
# # print ("download dir: ", download_dir )

# # print a nice tree
# # ! tree -d $download_dir

# ## if the above doesn't work, use the one below
# # print ("listing of download dir: ", os.listdir(os.path.dirname(data_location_local)))

# data_dir = os.path.join(os.path.dirname(data_location_local), 'horse-or-human')

# # peek inside data dir
# # ! tree -d $data_dir

# # print ('local data dir: ', data_dir)
# train_dir = os.path.join(data_dir, 'train')
# validation_dir = os.path.join(data_dir, 'validation')
# print ('train dir:', train_dir)
# print ('validation dir:', validation_dir)

## Step 3 : Prepare a Image Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rescale=1./255) # Generator for our validation data

data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              # class_mode='binary'
                                                              class_mode='categorical'
                                                             )

label_mappings = data_gen.class_indices
print ("label mappings : ", label_mappings)

In [ ]:
# x,y = next(data_gen)
# print (x)
# print (y)

## Step 4: Load the saved model

In [ ]:
from tensorflow.keras.models import load_model
import os

model_file = SAVED_MODEL_FILE
model_size_in_bytes = os.path.getsize(model_file)


model = load_model(model_file)

print ("Loaded model '{}',  size = {} bytes / {:,.1f} KB  / {:,.1f} MB".format(model_file, 
                                    model_size_in_bytes, model_size_in_bytes / 1024, 
                                    model_size_in_bytes / (1024*1024) ))



## Step 5: Predict

### 5.1 Predict on all validation/test images
Here is how we run prediction on all images in one shot!

In [ ]:
import numpy as np
from math import ceil


print ("predicting on {:,} test images".format(data_gen.n))
# we need a ceiling for steps
predictions = model.predict(data_gen, batch_size=data_gen.batch_size, 
                            steps=ceil(data_gen.n / data_gen.batch_size) )
print( 'predictions.shape: ', predictions.shape)

if data_gen.class_mode == 'categorical':
    # converting softmax --> classes
    print ("convering softmax --> classes")
    predictions2 = [ np.argmax(p) for p in predictions]

if data_gen.class_mode == 'binary':
    # converting sigmoid --> classes
    print ("converting sigmod --> binary")
    predictions2 = [0 if n < 0.5 else 1 for n in predictions]


# ## Ensure all predictions match
assert(len(predictions) == len(predictions2) == len(data_gen.classes) )

### 5.2 - Predict on a Single Image
Sometimes we just want to predict on certain images.  
Here is how.  
Note how we are loading images and shaping the image

In [ ]:
## Every time you run this cell, we will process a random image

import random
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# print ("image directory: ", data_gen.directory)
print ("label mappings: ", label_mappings)

index = random.randint(0, data_gen.n-1)

image_file = data_gen.filepaths[index]
print ("image file: ", image_file)
img = image.load_img(image_file, target_size = (IMG_WIDTH, IMG_HEIGHT))

# scale the data
img_data = image.img_to_array(img) / 255.0

## we need to shape the date to match the input shape network is expecting
# option 1 reshape
#img_data = np.expand_dims(img_data, axis = 0)
#prediction = model.predict (image_data)
# or option 2: no reshape and predict
prediction = model.predict(img_data[None]) # this is softmax output
print ('softmax output: ', prediction)

index_of_highest_probability = np.argmax(prediction[0])
value_of_highest_probability = prediction[0][index_of_highest_probability]

print ("predicted class: ", index_of_highest_probability, ", probability:", value_of_highest_probability)

## plot the image
plt.imshow(img_data)

### 5.3 - Predict on Custom Images
Let's predict on some of our 'own' images (images that are not part of training/validation set).  
The simplest approach is to copy image files into a directory.

- we have some sample images that were not part of the  dataset in subdirectories of `my-images` directory
- Download some pictures and put them into a a subdirectory of `my-images`  (e.g  `my-images/cat-dog`)
- And the following code will predict on them
- For fun, also include pictures of 'not seen' labels.  For example if the model trained on  cats-and-dog, feed the model 'human-or-horse' and see what happens :-) 

Here are some websites that offer free images:
- https://www.pexels.com/
- https://pixabay.com/

In [ ]:
import os, glob
from image_utils import   predict_on_images_in_dir, plot_image_predictions
import pprint

my_images_dir = 'my-images/horse-human'

prediction_results = predict_on_images_in_dir(model, my_images_dir, IMG_WIDTH, IMG_HEIGHT)
# pprint.pprint (prediction_results)

plot_image_predictions(prediction_results, "Predict on sample images", label_mappings)

## Step 6 - Explore Predictions on classes
This a fun experiment to visually see how our model is performing

In [ ]:
from image_utils import plot_prediction_stats_on_all_classes

data_gen.reset() # reset back to batch-1
plot_prediction_stats_on_all_classes(model, data_gen)